# Regressão Linear para Previsão de Preços de Imóveis

## 1. Objetivo do Estudo

Construir um modelo de regressão linear para prever o valor mediano das casas no dataset California Housing, avaliando a qualidade do modelo e a importância das variáveis.

## 2. Fonte & Licença

Dataset *California Housing* (scikit-learn). Licença: BSD-3 Clause.

## 3. Dicionário de Dados

- `MedInc`: Renda mediana.
- `HouseAge`: Idade mediana da casa.
- `AveRooms`: Média de quartos.
- `MedHouseVal`: Valor mediano da casa (alvo).

## 4. Metodologia

Carregamento do dataset California Housing. Análise de correlação. Ajuste do modelo de Regressão Linear Múltipla (OLS) usando `statsmodels`. Verificação dos pressupostos (linearidade, normalidade dos resíduos, homocedasticidade). Análise de multicolinearidade (VIF). Interpretação dos coeficientes e métricas do modelo.

## 5. Análise e Resultados

In [ ]:
# Imports iniciais
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from src.data.loaders import load_california_housing

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42) # Para reprodutibilidade

# 5.1. Carregar e Preparar os Dados
df = load_california_housing()

# Variável dependente (y) e independentes (X)
y = np.log(df['MedHouseVal']) # Usando log-transformação para normalizar a distribuição
X = df[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]

# Adicionar uma constante ao modelo (necessário para statsmodels)
X = sm.add_constant(X)

print("Primeiras 5 linhas das variáveis independentes (X):")
print(X.head())
print("
Primeiras 5 linhas da variável dependente (y):")
print(y.head())

# 5.2. Análise de Correlação
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação das Variáveis')
plt.show()

# 5.3. Ajuste do Modelo de Regressão Linear (OLS)
model = sm.OLS(y, X).fit()
print("
Resumo do Modelo de Regressão Linear:")
print(model.summary())

# 5.4. Verificação dos Pressupostos
# Resíduos
residuals = model.resid
fitted_values = model.fittedvalues

# Linearidade e Homocedasticidade (Resíduos vs. Valores Ajustados)
plt.figure(figsize=(14, 6))
sns.scatterplot(x=fitted_values, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Resíduos vs. Valores Ajustados')
plt.xlabel('Valores Ajustados')
plt.ylabel('Resíduos')
plt.show()

# Normalidade dos Resíduos (QQ-Plot)
plt.figure(figsize=(8, 8))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('QQ-Plot dos Resíduos')
plt.show()

# Teste de Normalidade dos Resíduos (Shapiro-Wilk)
shapiro_res = stats.shapiro(residuals)
print(f"
Teste de Shapiro-Wilk para Resíduos: W={shapiro_res.statistic:.3f}, p={shapiro_res.pvalue:.3f}")
if shapiro_res.pvalue > 0.05:
    print("  Não há evidências para rejeitar a hipótese nula de normalidade dos resíduos.")
else:
    print("  Há evidências para rejeitar a hipótese nula de normalidade dos resíduos.")

# 5.5. Análise de Multicolinearidade (VIF)
# Remover a constante antes de calcular o VIF
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("
Fatores de Inflação da Variância (VIF):")
print(vif_data.sort_values(by='VIF', ascending=False))


## 6. Interpretação

- **Resumo do Modelo**: A tabela de resumo do `statsmodels` fornece uma riqueza de informações, incluindo os coeficientes de regressão, seus p-valores, R-quadrado, R-quadrado ajustado e estatísticas de teste F.
- **Coeficientes**: Cada coeficiente indica a mudança esperada na variável dependente (log do valor da casa) para cada unidade de aumento na variável independente correspondente, mantendo as outras variáveis constantes. Coeficientes com p-valores baixos (tipicamente < 0.05) são considerados estatisticamente significativos.
- **R-quadrado e R-quadrado Ajustado**: O R-quadrado indica a proporção da variância na variável dependente que é explicada pelo modelo. O R-quadrado ajustado é uma versão modificada que penaliza a inclusão de preditores desnecessários, sendo mais útil para comparar modelos com diferentes números de preditores.
- **Pressupostos**: A verificação dos pressupostos é crucial para a validade do modelo:
  - **Linearidade e Homocedasticidade**: O gráfico de resíduos versus valores ajustados deve mostrar uma dispersão aleatória dos pontos em torno de zero, sem padrões visíveis (linearidade) e com uma variância constante (homocedasticidade). Padrões como funil ou curvas indicam violação.
  - **Normalidade dos Resíduos**: O QQ-Plot e o teste de Shapiro-Wilk ajudam a verificar se os resíduos seguem uma distribuição normal. Resíduos normais são um pressuposto para a inferência estatística (p-valores, intervalos de confiança).
- **Multicolinearidade (VIF)**: Valores de VIF acima de 5 ou 10 indicam multicolinearidade significativa, o que pode tornar os coeficientes de regressão instáveis e difíceis de interpretar. Nesses casos, pode ser necessário remover variáveis ou combiná-las.

## 7. Limitações

- A regressão linear assume uma relação linear entre as variáveis. Se a relação for não linear, o modelo pode não ser adequado.
- A qualidade do modelo depende da satisfação dos pressupostos. Violações severas podem levar a conclusões inválidas.
- A transformação logarítmica da variável dependente facilita a modelagem, mas a interpretação dos coeficientes na escala original requer exponenciação.
- O modelo não considera fatores externos não incluídos nas variáveis, que podem influenciar o preço das casas.

## 8. Reprodutibilidade

- **Semente Aleatória**: `np.random.seed(42)` foi utilizada para garantir a reprodutibilidade de quaisquer processos aleatórios.
- **Versões das Bibliotecas**: As versões das bibliotecas utilizadas podem ser verificadas executando `pip freeze` no ambiente do projeto.